# Day 8 - Seven Segment Search

https://adventofcode.com/2021/day/8

## Part 1

The first part is relatively easy, just determining how many output digits match the unique criteria for 7-segment digits `1`, `4`, `7`, and `8`. Just run through the inputs, count those strings in that part of the line, and produce a sum.

In [28]:
from pathlib import Path

INPUTS = Path("input.txt").read_text().strip().split("\n")


class InputLine:
    def __init__(self, line: str) -> None:
        self.line: str = line
        self.signals, self.digits = [x.strip().split() for x in line.split("|")]


all_lines = [InputLine(x) for x in INPUTS]


In [29]:
total = 0
for line in all_lines:
    total += sum([1 for x in line.digits if len(x) in (2, 3, 4, 7)])

print(f"Number of 1s, 4s, 7s, and 8s: {total}")

Number of 1s, 4s, 7s, and 8s: 303


## Part 2

This requires some puzzling to help lock down which signal corresponds to which segment.

```
 aaaa
b    c
b    c
 dddd
e    f
e    f
 gggg
```

This is the original. Of course, the real ones are random, so we can only use the above letters as a reference.

To lock down which is which requires process of elimination based on which 7-segment numbers must contain certain segments present in one of our known signals (1, 4, 7, 8). Mostly this depends on the contents of the `1` and the `4`:

- **9**: 6-segment signal containing every segment of the `4` (at least one is missing from either the `0` or `6`).
- **6**: 6-segment signal containing the 2 segments unique to the `4` (not present in the `1`).
- **0**: Last remaining 6-segment signal.
- **3**: 5-segment signal containing both segments of the `1`.
- **5**: 5-segment signal containing the unique segments of the `4`.
- **2**: Last remaining 5-segment signal.

With that determined, we can proceed to figure out which signal patterns pertain to which digit outputs. Because the signals and associated digits are also randomized strings, we need to be careful about sorting those inputs and output digits to ensure they match (although using sets would also help, I find working with the strings in mappings to be easier).

In retrospect, the solution for part 1 provided the hint that using the unique patterns would help in finding the rest. At first I thought this meant locking down specific segments to produce the real mapping of signal to segment, but as shown above, that's not necessary.

In [30]:
class InputLinePart2(InputLine):
    def __init__(self, line: str) -> None:
        super().__init__(line)
        self.signal_map: dict[str, int] = None

    def process_signals(self) -> None:
        signal_sets = list(map(set, self.signals))
        # Get our obvious ones settled
        signal_map = {}
        for signal in signal_sets:
            match len(signal):
                case 2:
                    signal_map[1] = signal
                case 3:
                    signal_map[7] = signal
                case 4:
                    signal_map[4] = signal
                case 7:
                    signal_map[8] = signal
    
        # determine the unique set of segments in the `4` not part of the `4`
        unique_to_four = signal_map[4] - signal_map[1]

        # 9: 6-segment signal containing all the same segments as `4`
        sixers = [x for x in signal_sets if len(x) == 6]
        signal_map[9] = [x for x in sixers if x & signal_map[4] == signal_map[4]][0]

        # 6: 6-segment containing the two segments unique to `4`, but not to `1`
        signal_map[6] = [x for x in sixers if x & unique_to_four == unique_to_four and x & signal_map[1] != signal_map[1]][0]

        # 0: remaining 6-segment signal.
        signal_map[0] = [x for x in sixers if x not in (signal_map[6], signal_map[9])][0]

        # 3: 5-segment containing all parts of `1`
        fivers = [x for x in signal_sets if len(x) == 5]
        signal_map[3] = [x for x in fivers if x & signal_map[1] == signal_map[1]][0]

        # 5: 5-segment containing the unique parts of `4`
        signal_map[5] = [x for x in fivers if x & unique_to_four == unique_to_four][0]

        # 2: Remaining 5-segment
        signal_map[2] = [x for x in fivers if x not in (signal_map[3], signal_map[5])][0]

        # Invert the map to convert from a signal string to the integer value
        # Key strings will be sorted
        self.signal_map = {''.join(sorted(v)): k for k, v in signal_map.items()}
    
    @property
    def number(self) -> int:
        if self.signal_map is None:
            self.process_signals()
        output = ''
        for digit in self.digits:
            key = ''.join(sorted(digit))
            output += str(self.signal_map[key])
        return int(output)


Before continuing, we'll run some tests for sanity, based on the example given on the AoC site:

In [31]:
def test_processing():
    sample_line = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf'
    thingy = InputLinePart2(sample_line)
    thingy.process_signals()
    expected = {
        'acedgfb': 8,
        'cdfbe': 5,
        'gcdfa': 2,
        'fbcad': 3,
        'dab': 7,
        'cefabd': 9,
        'cdfgeb': 6,
        'eafb': 4,
        'cagedb': 0,
        'ab': 1,
    }
    expected = {''.join(sorted(k)): v for k, v in expected.items()}
    for key, val in thingy.signal_map.items():
        assert expected[''.join(sorted(val))] == key


def test_number():
    sample_line = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf'
    thingy = InputLinePart2(sample_line)
    assert thingy.number == 5353

In [32]:
all_nums = []
for line in INPUTS:
    all_nums.append(InputLinePart2(line).number)

print(f"Sum of all outputs: {sum(all_nums)}")

Sum of all outputs: 961734
